In [63]:
import soynlp
soynlp.__version__

'0.0.46'

## Corpus

In [80]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = '../data/md_contactus_question_spacing.txt'
corpus = DoublespaceLineCorpus(corpus_fname, iter_sent=True, num_sent=4)

for n_sent, sent in enumerate(corpus):
    print('sent %d: %s %s'% (n_sent, sent[:50], '' if len(sent) < 50 else '...'))

sent 0: 추천인아이디요. . 추천인정보에 인스타아이디를 넣는건가요 아니면 로그인할때 메일주소를 넣는 ...
sent 1: 포스트 금액 문의 안녕하세요. 지금 신청은 했는데요. 제가 처음이라.. 잘 몰라서요. 포스 ...
sent 2: 제가 페이스북 페이지가 있는데 어떻게 등록하나요? 알려주세요 
sent 3: 페이지관리자입니다. 팔로우 1048명 페이지관리자인데 등록하고싶은데 등록이 안되서요... 


In [81]:
corpus = DoublespaceLineCorpus(corpus_fname, iter_sent=False, num_doc=3)

for n_doc, doc in enumerate(corpus):
    print('docs %d (#sents= %d): %s %s'% (n_doc, len(doc.split('  ')), doc[:50].strip(), '' if len(doc) < 50 else '...'))

docs 0 (#sents= 1): 추천인아이디요. . 추천인정보에 인스타아이디를 넣는건가요 아니면 로그인할때 메일주소를 넣는 ...
docs 1 (#sents= 1): 포스트 금액 문의 안녕하세요. 지금 신청은 했는데요. 제가 처음이라.. 잘 몰라서요. 포스 ...
docs 2 (#sents= 1): 제가 페이스북 페이지가 있는데 어떻게 등록하나요? 알려주세요 


## WordExtractor (Cohesion score, Branching Entropy, Accessor Variety)

In [82]:
from soynlp.word import WordExtractor

corpus = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
word_extractor = WordExtractor(left_max_length=10,
                               right_max_length=6,
                               min_count=100)
word_extractor.train(corpus)

training was done. used memory 0.231 Gb


In [83]:
words = word_extractor.word_scores()

all cohesion probabilities was computed. # words = 348
all branching entropies was computed # words = 12214
all accessor variety was computed # words = 12214


In [84]:
len(words)

550

In [85]:
words['환급']

Scores(cohesion_forward=0.9766990291262136, cohesion_backward=0, left_branching_entropy=3.1200165166238483, right_branching_entropy=3.176185129512808, left_accessor_variety=45, right_accessor_variety=34, leftside_frequency=503, rightside_frequency=0)

In [86]:
words['환급'].cohesion_forward

0.9766990291262136

In [87]:
for word in ['환급', '환급금']:
    print(word, '(%.3f, %.3f)' % (scores[word].left_branching_entropy,
                                  scores[word].right_branching_entropy))

환급 (3.044, 2.795)
환급금 (0.000, 0.000)


In [88]:
def word_score(score):
    import math
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:100]:
    print('%s\t\t\t(%d, %.3f, %.3f)' % (word, 
                                   score.leftside_frequency, 
                                   score.cohesion_forward,
                                   score.right_branching_entropy
                                  ))

단어   (빈도수, cohesion, branching entropy)

ㅠㅠ			(528, 0.887, 4.111)
했는데			(244, 0.774, 4.029)
^^			(104, 0.945, 3.816)
혹시			(273, 0.945, 3.721)
ㅜㅜ			(132, 0.857, 3.663)
합니다			(120, 0.943, 3.538)
캠페인			(1266, 0.981, 3.337)
제가			(524, 0.378, 4.257)
드립니다.			(155, 0.764, 3.470)
포스팅			(840, 0.696, 3.556)
환급			(503, 0.977, 3.176)
미디언스			(218, 0.841, 3.153)
감사합니다.			(134, 0.835, 3.059)
드립니다			(238, 0.806, 3.073)
언제			(377, 0.969, 2.862)
영상			(161, 0.834, 3.008)
변경			(326, 0.959, 2.830)
따로			(127, 0.864, 2.890)
안녕하세요.			(183, 0.586, 3.275)
안녕하세요			(821, 0.746, 3.016)
업로드			(322, 0.871, 2.767)
부탁드려요			(145, 0.654, 3.019)
취소			(652, 0.991, 2.592)
오늘			(331, 0.415, 3.413)
되나요?			(104, 0.459, 3.309)
승인			(407, 0.998, 2.465)
잘못			(302, 0.904, 2.555)
관련			(349, 0.864, 2.556)
하는데			(163, 0.354, 3.436)
채널			(212, 0.951, 2.414)
광고			(161, 0.976, 2.326)
삭제			(158, 1.000, 2.300)
문의			(1368, 0.755, 2.578)
등록			(717, 0.920, 2.334)
완료			(164, 0.970, 2.275)
수정			(451, 0.614, 2.715)
있는데			(122, 0.328, 3.338)
요청

## Noun extraction

In [89]:
from soynlp.noun import LRNounExtractor
from soynlp.noun import NewsNounExtractor

corpus = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
noun_extractor = NewsNounExtractor()
nouns = noun_extractor.train_extract(corpus)

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/mediance_ssh/Anaconda3/lib/site-packages/soynlp
scan vocabulary ... 
done (Lset, Rset, Eojeol) = (60666, 39366, 25849)
predicting noun score was done                                        
before postprocessing 8736
_noun_scores_ 1649
checking hardrules ... done
after postprocessing 1018
extracted 25 compounds from eojeols

In [90]:
for word in ['환급', '환급금', '모바일로', '캠페인을', '포스팅', '포스트', '캐시한급받고싶어요', '죄송하단']:
    print('%s is noun? %r\n' % (word, word in nouns))

환급 is noun? True

환급금 is noun? False

모바일로 is noun? False

캠페인을 is noun? False

포스팅 is noun? True

포스트 is noun? True

캐시한급받고싶어요 is noun? False

죄송하단 is noun? False



## Tokenization

In [91]:
from soynlp.tokenizer import LTokenizer,MaxScoreTokenizer, RegexTokenizer
cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
ltokenizer = LTokenizer(scores=cohesion_scores)

In [92]:
ltokenizer.tokenize('어느순간 사진이 삭제되있더라구요 그래서 재등록하긴했는데 이럴땐어떡게해야하나여')

['어느',
 '순간',
 '사진',
 '이',
 '삭제',
 '되있더라구요',
 '그래서',
 '재등록',
 '하긴했는데',
 '이럴땐',
 '어떡게해야하나여']

In [93]:
ltokenizer.tokenize('어느순간 사진이 삭제되있더라구요 그래서 재등록하긴했는데 이럴땐어떡게해야하나여', flatten=False)

[('어느', '순간'),
 ('사진', '이'),
 ('삭제', '되있더라구요'),
 ('그래서', ''),
 ('재등록', '하긴했는데'),
 ('이럴땐', '어떡게해야하나여')]

In [94]:
ltokenizer.tokenize('어느순간 사진이 삭제되있더라구요 그래서 재등록하긴했는데 이럴땐어떡게해야하나여', remove_r=True)

['어느', '사진', '삭제', '그래서', '재등록', '이럴땐']

In [98]:
maxscoretokenizer = MaxScoreTokenizer(scores=cohesion_scores)
maxscoretokenizer.tokenize('어느순간 사진이 삭제되있더라구요 그래서 재등록하긴했는데 이럴땐어떡게해야하나여'.replace(' ',''))

['어느',
 '순간',
 '사진',
 '이',
 '삭제',
 '되',
 '있더라구요',
 '그래서',
 '재',
 '등록',
 '하긴',
 '했는데',
 '이럴땐',
 '어떡',
 '게',
 '해야하나',
 '여']

In [153]:

regextokenizer = RegexTokenizer() # regexTokenizer
from soynlp.tokenizer import normalize # nomalize

corpus_fname = '../data/md_contactus_question_spacing.txt'
ary = []

with open(corpus_fname, encoding='utf-8') as f:
    for i,sent in enumerate(f):        
        if (i==10): break        
                
        # regexTokenizer
        text=''
        for word in regextokenizer.tokenize(sent):            
            text += word+' '
        sent = text
        
        # 노멀라이징        
        text=''
        for word in normalize(sent, n_repeat=2):
            text += word
        sent = text
        
        # 토크나이징
#         text=''
#         for word in ltokenizer.tokenize(sent, remove_r=False):
#         for word in maxscoretokenizer.tokenize(sent.replace(' ','')):
#             text += word+' '        
        
        ary.append(text)

ary

['추천인아이디요 . . 추천인정보에 인스타아이디를 넣는건가요 아니면 로그인할때 메일주소를 넣는건가요 ? 제가 소개해서 가입하신분들이 인스타아이디 dlsks gml 01 로 추천인을 넣으셨다는데 포인트정보가 없어서요',
 '포스트 금액 문의 안녕하세요 . 지금 신청은 했는데요 . 제가 처음이라 .. 잘 몰라서요 . 포스트 금액을 제가 설정하는 것이던데 , 제가 받고 싶은 금액을 적으면 되는것인지 .. 아님 당첨되면 회사에서 정한 금액을 받는 것인지 궁금해요 . 제가 금액을 높게 신청하면 컨텍이 안되는 것인지 .. 궁금하네요',
 '제가 페이스북 페이지가 있는데 어떻게 등록하나요 ? 알려주세요',
 '페이지관리자입니다 . 팔로우 1048 명 페이지관리자인데 등록하고싶은데 등록이 안되서요 ...',
 '페이지주소입니다 . https :// www . facebook . com / ukeukeuke 18',
 '정진우싱글앨범 문의요 싱금앨범을 보내주면 포스트하는거 아닌가요 ? 아니면 인스타에 올리 사진이나 동영상은 어디서받아야되나요 ? 메일로 보내주시는건가요 ?',
 '방금캠페인 포인트를 15 만포인트 신청햇는데 그럼 이거 포스팅해주면 몇 포인트가 들어온다는거죠 ?',
 '네 자세한 답변감사하구요 결론적으로 이번 캠페인은 입찰금액이 들어오는게 아닌 리워드로 받은것하고 1000 원 포인트만 들어온다는 이야기시죠 ? 처음이라서요',
 '제휴문의 인스타 시스템을 운영 하고 있습니다 .. 효과적인 아이디어가 있어 상호 윈윈 할만한 수익건이라 생각되어 연락 드립니다 .. 연락 한번 부탁 합니다 .. 010 4871 9487 이승우 대표 입니다',
 '제휴 관련 회신입니다 .. 회신 메일은 잘받았습니다 . 혹시 연락척 없던데 담당자분 연락처 없어서 혹시 빠지신게 아닌가 해서요 .. 자료는 보내 드릴수 있지만 좀 더 매력 있는 제안으로 협력 하고 싶어서 그렇습니다 ..']

In [131]:
# writedata.py
f = open("../data/md_contactus_question_tokenized.txt", 'w', encoding='utf-8')
for i,sent in enumerate(ary):
    data = ary[i]+"\n"
    f.write(data)
f.close()

In [ ]:
from collections import Counter
from konlpy.tag import Twitter
twitter = Twitter()

In [173]:
## twitter tokenizer
corpus_fname = '../data/md_contactus_question_spacing.txt'
ary = []

with open(corpus_fname, encoding='utf-8') as f:
    for i,sent in enumerate(f):
        if(i==1): break
        text = ''
        for word in twitter.morphs(sent):
            text += word+" "
        ary.append(text)
        text = ''
        for word in twitter.nouns(sent):
            text += word+" "
        ary.append(text)

ary        


['추천 인 아이디 요 . . 추천 인 정보 에 인스타 아이디 를 넣는 건 가요 아니 면 로그 인할 때 메일 주소 를 넣는 건 가요 ? \xa0 제 가 소개해서 가입하신 분 들 이 인스타 아이디 dlsksgml 01 로 추천 인 을 넣으 셨 다는 데 포인트 정보 가 없어 서요 ',
 '추천 아이디 추천 정보 인스타 아이디 가요 로그 때 메일 주소 가요 제 분 인스타 아이디 로 추천 셨 포인트 정보 서요 ']

In [ ]:
## twitter tokenizer & word counter

In [174]:
corpus_fname = '../data/md_contactus_question_spacing.txt'
ary = []

with open(corpus_fname, encoding='utf-8') as f:
    for i,sent in enumerate(f):
        ary.append(twitter.nouns(sent))

flatten_docs = [word for sent in ary for word in sent]
count = Counter(flatten_docs)
print(count.most_common(20))

[('문의', 1625), ('캠페인', 1324), ('포스팅', 893), ('제', 662), ('신청', 646), ('확인', 633), ('포인트', 609), ('제품', 591), ('등록', 584), ('취소', 566), ('포스트', 544), ('배송', 514), ('환급', 509), ('일', 505), ('수정', 481), ('아직', 432), ('승인', 432), ('캐시', 428), ('변경', 427), ('가요', 425)]


In [175]:
# 임베딩 후 시각화... 

In [ ]:
import os
    
class CommentWord2Vec:
    
    def __init__(self, fname):
        self.fname = fname
        if not os.path.exists(fname):
            print('File not found: %s' % fname)
        
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                movie_idx, text, score = doc.split('\t')
                yield text.split()
                
                
word2vec_corpus = CommentWord2Vec(tokenized_corpus_fname)

for num_doc, doc in enumerate(word2vec_corpus):
    if num_doc > 5: break
    print(doc)